In [ ]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add 'TSE3D' folder to current working path in order to access the functions
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/TSE3D")

# Overview

In the following notebook we compute the TRA on the three-dimensional, steady ABC-flow. The velocity field is analytically defined. The notebook is structured as follows:

1. Define analytic velocity-field of steady ABC-flow.
<br />
2. Define computational parameters (such as the number of cores).
 <br />
3. Define spatio-temporal domain.
 <br />
4. Define integration scheme for computing flow map (=trajectories) and derivative of flow map (=velocities).
 <br />
5. TRA:

    * Compute velocity along trajectories $ \lbrace \mathbf{\dot{x}}(t_{i}) \rbrace_{i=0}^N $ over the time interval $ [t_0, t_N] $.
    <br />
    
    * Compute $ \mathrm{\overline{TSE}} $/  $ \mathrm{TSE} $ from velocity along trajectories:
    
    \begin{equation}
    \mathrm{\overline{TSE}}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \sum_{i = 0}^{N-1} \cos^{-1}\dfrac{\langle \mathbf{\dot{x}}(t_i), \mathbf{\dot{x}}(t_{i+1}) \rangle+\mathrm{v}_0^2}{\sqrt{|\mathbf{\dot{x}}(t_i)|^2+\mathrm{v}_0^2}\sqrt{|\mathbf{\dot{x}}(t_{i+1})|^2+\mathrm{v}_0^2}}
    \end{equation}
    
    \begin{equation}
    \mathrm{\overline{TSE}}_{t_0}^{t_N} = \dfrac{1}{t_N-t_0} \sum_{i=0}^{N-1}|\log{\dfrac{\sqrt{|\mathbf{\dot{x}}(t_{i+1})|^2 +\mathrm{v}_0^2}}{\sqrt{|\mathbf{\dot{x}}(t_i)|^2 +\mathrm{v}_0^2}}}|
    \end{equation}
    
    $ \mathrm{v}_0 $ is a characteristic velocity. In a steady ABC-flow, $ \mathrm{v}_0 = 0 $.

# Define analytic velocity field

Here we define the analytic steady velocity of the ABC-flow.

\begin{equation}
\begin{pmatrix} \dot{x}(t) \\ \dot{y}(t) \\ \dot{z}(t) \end{pmatrix} = \begin{pmatrix} \sqrt{3}\sin(z)+\cos(y) \\ \sqrt{2}\sin(x)+\sqrt{3}\cos(z) \\ \sin(y)+\sqrt{2}\cos(x) \end{pmatrix}, 
\end{equation} with $ \nu = 0.01 $.

In [ ]:
def velocity(x, y, z):

    u = sqrt(3)*np.sin(z)+np.cos(y)
    v = sqrt(2)*np.sin(x)+sqrt(3)*np.cos(z)
    w = np.sin(y)+sqrt(2)*np.cos(x)
    
    return np.array([u, v, w])

# Computational parameters

Here we define the computational parameters.

In [ ]:
# import numpy
import numpy as np

# import math tools
from math import sqrt, pi

# number of cores to be used for parallel computing
Ncores = 4

# periodic boundary conditions
periodic_x = True
periodic_y = True
periodic_z = True
periodic = [periodic_x, periodic_y, periodic_z]

# Characteristic velocity
v0 = 0 # ABC flow is dimensionless

# Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [ ]:
%%time
# Initial time
t0 = 0 # float

# Final time
tN = 50 # float

# Time step-size
dt = 0.05 # float

# NOTE: For computing the backward FTLE field tN < t0 and dt < 0.

time = np.arange(t0, tN+dt, dt) # shape (Nt,)

# Length of time interval
lenT = abs(tN-t0) # float

# boundaries
xmin = 0 # float
xmax = 2*np.pi # float
ymin = 0 # float
ymax = 2*np.pi # float
zmin = 0 # float
zmax = 2*np.pi # float

# Resolution of meshgrid
Ny = 200 # int
Nx = 200 # int
Nz = 200 # int

x_domain = np.linspace(xmin, xmax, Nx, endpoint = True) # array (Nx, )
y_domain = np.linspace(ymin, ymax, Ny, endpoint = True) # array (Ny, )
z_domain = np.linspace(zmin, zmax, Nz, endpoint = True) # array (Nz, )

dx = x_domain[1]-x_domain[0] # float
dy = y_domain[1]-y_domain[0] # float
dz = z_domain[1]-z_domain[0] # float

X_domain, Y_domain, Z_domain = np.meshgrid(x_domain, y_domain, z_domain) # array (Ny, Nx, Nz)

Ny = X_domain.shape[0] # int
Nx = X_domain.shape[1] # int
Nz = X_domain.shape[2] # int

# Define Integration

In [ ]:
def integration_dFdt(time, x):
    
    # reshape x
    x = x.reshape(3, -1) # reshape array (3, Nx*Ny*Nz)
    
    # Initialize arrays for flow map and derivative of flow map
    Fmap = np.zeros((len(time), 3, x.shape[1])) # array (Nt, 3, Nx*Ny*Nz)
    dFdt = np.zeros((len(time)-1, 3, x.shape[1])) # array (Nt-1, 3, Nx*Ny*Nz)
    
    # Step-size
    dt = time[1]-time[0] # float
    
    counter = 0 # int

    # initial conditions
    Fmap[counter,:,:] = x
    
    # Runge Kutta 4th order integration with fixed step size dt
    for t in time[:-1]:
        
        if t%1 == 0:
            print('Percentage completed: ', np.around(t/time[-1], 3))
        
        Fmap[counter+1,:, :], dFdt[counter,:,:] = RK4_classic_step(t, Fmap[counter,:, :], dt)[:2]
    
        counter += 1
    
    return Fmap, dFdt

def RK4_classic_step(t, x1, dt):
    
    t0 = t # float
    
    # Compute x_prime at the beginning of the time-step by re-orienting and rescaling the vector field
    x_prime = velocity(x1[0,:], x1[1,:], x1[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k1 = dt * x_prime # array(3, Nx*Ny*Nz)
    
    # Update position at the first midpoint.
    x2 = x1 + .5 * k1 # array(3, Nx*Ny*Nz)
     
    # Update time
    t = t0 + .5*dt # float
    
    # Compute x_prime at the first midpoint.
    x_prime = velocity(x2[0,:], x2[1,:], x2[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k2 = dt * x_prime # array(3, Nx*Ny*Nz)

    # Update position at the second midpoint.
    x3 = x1 + .5 * k2 # array(3, Nx*Ny*Nz)
    
    # Update time
    t = t0 + .5*dt # float
    
    # Compute x_prime at the second midpoint.
    x_prime = velocity(x3[0,:], x3[1,:], x3[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k3 = dt * x_prime # array(3, Nx*Ny*Nz)
    
    # Update position at the endpoint.
    x4 = x1 + k3 # array(3, Nx*Ny*Nz)
    
    # Update time
    t = t0+dt # float
    
    # Compute derivative at the end of the time-step.
    x_prime = velocity(x4[0,:], x4[1,:], x4[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k4 = dt * x_prime
    
    # Compute RK4 derivative
    y_prime_update = 1.0 / 6.0*(k1 + 2 * k2 + 2 * k3 + k4) # array(3, Nx*Ny*Nz)
    
    # Integration y <-- y + y_primeupdate
    y_update = x1 + y_prime_update # array(3, Nx*Ny*Nz)
    
    return y_update, y_prime_update/dt

# $ \mathrm{\overline{TSE}} $

The computation of the $ \mathrm{\overline{TSE}} $ is not done over the 3D meshgrid but only over the 2D faces of the cube $ [0,2\pi]^3 $.

## Define function to compute $ \mathrm{\overline{TSE}} $ in parallel.

In [ ]:
%%time

# Import function to compute Trajectory Rotation Average (TRA)
from ipynb.fs.defs.TSE import _TSE, _TSE_bar

# Import package for parallel computing
from joblib import Parallel, delayed

def compute_TSE(x0, y0, z0):
    
    # Set initial conditions
    X0 = np.array([x0, y0, z0]) # array (3, Nx*Ny*Nz)
    
    # Velocity along particle trajectories
    DFdt = integration_dFdt(time, X0)[1] # array (Nt, 3, Nx*Ny*Nz)
    
    # Compute TSE, TSE_bar
    TSE, TSE_bar = [], []
    
    for i in range(DFdt.shape[2]):
        TSE.append(_TSE(DFdt[:,:,i].transpose(), lenT, v0))
        TSE_bar.append(_TSE_bar(DFdt[:,:,i].transpose(), lenT, v0))

    return TSE, TSE_bar

# Split x0,y0,z0 into 'Ncores' equal batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

## XY-plane

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )

# XY-plane
x0_xy = X_domain[:,:,-1].ravel() # array (Nx*Ny*Nz, )
y0_xy = Y_domain[:,:,-1].ravel() # array (Nx*Ny*Nz, )
z0_xy = Z_domain[:,:,-1].ravel() # array (Nx*Ny*Nz, )

x0_batch = list(split(x0_xy, Ncores)) # list (Nx*Ny*Nz)
y0_batch = list(split(y0_xy, Ncores)) # list (Nx*Ny*Nz)
z0_batch = list(split(z0_xy, Ncores)) # list (Nx*Ny*Nz)

# Parallel computing
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_TSE)(x0_batch[i], y0_batch[i], z0_batch[i]) for i in range(len(x0_batch)))

# Extract results of omega computation on xy-plane
TSE_xy = results[0][0]
TSE_bar_xy = results[0][1]
for res in results[1:]:
    TSE_xy = np.append(TSE_xy, res[0], axis = -1)
    TSE_bar_xy = np.append(TSE_bar_xy, res[1], axis = -1)

# Reshape arrays
X0_xy = np.array(x0_xy).reshape(Ny,Nx) # array (Ny, Nx)
Y0_xy = np.array(y0_xy).reshape(Ny,Nx) # array (Ny, Nx)
Z0_xy = np.array(z0_xy).reshape(Ny,Nx) # array (Ny, Nx)
TSE_xy = np.array(TSE_xy).reshape(Ny,Nx) # array (Ny, Nx)
TSE_bar_xy = np.array(TSE_bar_xy).reshape(Ny,Nx) # array (Ny, Nx)

## XZ-plane

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )

# XZ-plane
x0_xz = X_domain[0,:,:].ravel() # array (Nx*Ny*Nz, )
y0_xz = Y_domain[0,:,:].ravel() # array (Nx*Ny*Nz, )
z0_xz = Z_domain[0,:,:].ravel() # array (Nx*Ny*Nz, )

x0_batch = list(split(x0_xz, Ncores)) # list (Nx*Ny*Nz)
y0_batch = list(split(y0_xz, Ncores)) # list (Nx*Ny*Nz)
z0_batch = list(split(z0_xz, Ncores)) # list (Nx*Ny*Nz)

# Parallel computing
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_TSE)(x0_batch[i], y0_batch[i], z0_batch[i]) for i in range(len(x0_batch)))

# Extract results of omega computation on xz-plane
TSE_xz = results[0][0]
TSE_bar_xz = results[0][1]
for res in results[1:]:
    TSE_xz = np.append(TSE_xz, res[0], axis = -1)
    TSE_bar_xz = np.append(TSE_bar_xz, res[1], axis = -1)
    
# Reshape arrays
X0_xz = np.array(x0_xz).reshape(Nz,Nx) # array (Nz, Nx)
Y0_xz = np.array(y0_xz).reshape(Nz,Nx) # array (Nz, Nx)
Z0_xz = np.array(z0_xz).reshape(Nz,Nx) # array (Nz, Nx)
TSE_xz = np.array(TSE_xz).reshape(Nz,Nx) # array (Nz, Nx)
TSE_bar_xz = np.array(TSE_bar_xz).reshape(Nz,Nx) # array (Nz, Nx)

## YZ-plane

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )

# YZ-plane
x0_yz = X_domain[:,0,:].ravel() # array (Nx*Ny*Nz, )
y0_yz = Y_domain[:,0,:].ravel() # array (Nx*Ny*Nz, )
z0_yz = Z_domain[:,0,:].ravel() # array (Nx*Ny*Nz, )
    
x0_batch = list(split(x0_yz, Ncores)) # list (Nx*Ny*Nz)
y0_batch = list(split(y0_yz, Ncores)) # list (Nx*Ny*Nz)
z0_batch = list(split(z0_yz, Ncores)) # list (Nx*Ny*Nz)

# Parallel computing
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_TSE)(x0_batch[i], y0_batch[i], z0_batch[i]) for i in range(len(x0_batch)))

# Extract results of omega computation on yz-plane
TSE_yz = results[0][0]
TSE_bar_yz = results[0][1]
for res in results[1:]:
    TSE_yz = np.append(TSE_yz, res[0], axis = -1)
    TSE_bar_yz = np.append(TSE_bar_yz, res[1], axis = -1)

# Reshape arrays
X0_yz = np.array(x0_yz).reshape(Nz,Ny) # array (Nt, Nz, Ny)
Y0_yz = np.array(y0_yz).reshape(Nz,Ny) # array (Nt, Nz, Ny)
Z0_yz = np.array(z0_yz).reshape(Nz,Ny) # array (Nt, Nz, Ny)
TSE_yz = np.array(TSE_yz).reshape(Nz,Ny) # array (Nz, Ny)
TSE_bar_yz = np.array(TSE_bar_yz).reshape(Nz,Ny) # array (Nz, Ny)

In [ ]:
# Import plotly for 3D figures
from plotly import graph_objs as go

# define minimum and maximum values for colorbar
min_value = min(np.min(TSE_yz),min(np.min(TSE_xy), np.min(TSE_xz)))
max_value = max(np.max(TSE_yz),max(np.max(TSE_xy), np.max(TSE_xz)))

# create figure
fig = go.Figure(data=[go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy, surfacecolor = TSE_xy, showscale = True, colorscale='rainbow', cmin=min_value, cmax=max_value)])
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz, z = Z0_xz, surfacecolor = TSE_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy-2*pi, surfacecolor = TSE_xy, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz+2*pi, z = Z0_xz, surfacecolor = TSE_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz+2*pi, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))

# camera
camera = dict(eye=dict(x=-1.45,y=-1.45,z=1.7))
fig.update_layout(scene_camera=camera)

fig.show()

In [ ]:
# Import plotly for 3D figures
from plotly import graph_objs as go

# define minimum and maximum values for colorbar
min_value = min(np.min(TSE_bar_yz),min(np.min(TSE_bar_xy), np.min(TSE_bar_xz)))
max_value = max(np.max(TSE_bar_yz),max(np.max(TSE_bar_xy), np.max(TSE_bar_xz)))

# create figure
fig = go.Figure(data=[go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy, surfacecolor = TSE_bar_xy, showscale = True, colorscale='rainbow', cmin=min_value, cmax=max_value)])
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz, z = Z0_xz, surfacecolor = TSE_bar_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy-2*pi, surfacecolor = TSE_bar_xy, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz+2*pi, z = Z0_xz, surfacecolor = TSE_bar_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz+2*pi, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))

# camera
camera = dict(eye=dict(x=-1.45,y=-1.45,z=1.7))
fig.update_layout(scene_camera=camera)

fig.show()

Hyperbolic LCS are visible as fronts in the $ \mathrm{\overline{TSE}}_{t_0}^{t_N} $ and ridges/trenches in the $ \mathrm{TSE}_{t_0}^{t_N} $. In order to test the ability of the $ \mathrm{\overline{TSE}} $ to extract hyperbolic flow regions even in a sparse data setting, we randomly progressively downsample the trajectory data. As shown in the following figures, $ \mathrm{\overline{TSE}} $ is able to capture the major fronts even at very low resolutions.

In [ ]:
# import random number generator
import random

# 10% subsampling
rdn10 = random.sample(range(0, Nx*Ny), int(Nx*Ny/10))

# Subsample initial grid and TSE_bar field (=subsample trajectories) on XY-plane
x_XY_subsampled10 = X0_xy.ravel()[rdn10] # array (Nx*Ny/10, )
y_XY_subsampled10 = Y0_xy.ravel()[rdn10] # array (Nx*Ny/10, )
TSE_bar_XY_subsampled10 = TSE_bar_xy.ravel()[rdn10] # array (Nx*Ny/10, )

# Subsample initial grid and TSE_bar field (=subsample trajectories) on XZ-plane
x_XZ_subsampled10 = X0_xz.ravel()[rdn10] # array (Nx*Ny/10, )
y_XZ_subsampled10 = Z0_xz.ravel()[rdn10] # array (Nx*Ny/10, )
TSE_bar_XZ_subsampled10 = TSE_bar_xz.ravel()[rdn10] # array (Nx*Ny/10, )

# Subsample initial grid and TSE_bar field (=subsample trajectories) on YZ-plane
x_YZ_subsampled10 = Y0_yz.ravel()[rdn10] # array (Nx*Ny/10, )
y_YZ_subsampled10 = Z0_yz.ravel()[rdn10] # array (Nx*Ny/10, )
TSE_bar_YZ_subsampled10 = TSE_bar_yz.ravel()[rdn10] # array (Nx*Ny/10, )

# Import interpolant for scatter interpolation
from scipy.interpolate import Rbf

# Compute Interpolant
interp10_XY = Rbf(x_XY_subsampled10, y_XY_subsampled10, TSE_bar_XY_subsampled10, function = "linear")
interp10_XZ = Rbf(x_XZ_subsampled10, y_XZ_subsampled10, TSE_bar_XZ_subsampled10, function = "linear")
interp10_YZ = Rbf(x_YZ_subsampled10, y_YZ_subsampled10, TSE_bar_YZ_subsampled10, function = "linear")

# Re-evaluate interpolant over original (initial) meshgrid
TSE_bar_interpolated10_xy = interp10_XY(X0_xy, Y0_xy) 
TSE_bar_interpolated10_xz = interp10_XZ(X0_xz, Z0_xz) 
TSE_bar_interpolated10_yz = interp10_YZ(Y0_yz, Z0_yz)

In [ ]:
# define minimum and maximum values for colorbar
min_value = min(np.min(TSE_bar_yz),min(np.min(TSE_bar_xy), np.min(TSE_bar_xz)))
max_value = max(np.max(TSE_bar_yz),max(np.max(TSE_bar_xy), np.max(TSE_bar_xz)))

# create figure
fig = go.Figure(data=[go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy, surfacecolor = TSE_bar_interpolated10_xy, showscale = True, colorscale='rainbow', cmin=min_value, cmax=max_value)])
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz, z = Z0_xz, surfacecolor = TSE_bar_interpolated10_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_interpolated10_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy-2*pi, surfacecolor = TSE_bar_interpolated10_xy, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz+2*pi, z = Z0_xz, surfacecolor = TSE_bar_interpolated10_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz+2*pi, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_interpolated10_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))

# camera
camera = dict(eye=dict(x=-1.45,y=-1.45,z=1.7))
fig.update_layout(scene_camera=camera)

fig.show()

In [ ]:
# 1% subsampling
rdn100 = random.sample(range(0, Nx*Ny), int(Nx*Ny/100))

# Subsample initial grid and TSE_bar field (=subsample trajectories) on XY-plane
x_XY_subsampled100 = X0_xy.ravel()[rdn100] # array (Nx*Ny/10, )
y_XY_subsampled100 = Y0_xy.ravel()[rdn100] # array (Nx*Ny/10, )
TSE_bar_XY_subsampled100 = TSE_bar_xy.ravel()[rdn100] # array (Nx*Ny/10, )

# Subsample initial grid and TRA field (=subsample trajectories) on XZ-plane
x_XZ_subsampled100 = X0_xz.ravel()[rdn100] # array (Nx*Ny/10, )
y_XZ_subsampled100 = Z0_xz.ravel()[rdn100] # array (Nx*Ny/10, )
TSE_bar_XZ_subsampled100 = TSE_bar_xz.ravel()[rdn100] # array (Nx*Ny/10, )

# Subsample initial grid and TRA field (=subsample trajectories) on YZ-plane
x_YZ_subsampled100 = Y0_yz.ravel()[rdn100] # array (Nx*Ny/10, )
y_YZ_subsampled100 = Z0_yz.ravel()[rdn100] # array (Nx*Ny/10, )
TSE_bar_YZ_subsampled100 = TSE_bar_yz.ravel()[rdn100] # array (Nx*Ny/10, )

# Compute Interpolant
interp100_XY = Rbf(x_XY_subsampled100, y_XY_subsampled100, TSE_bar_XY_subsampled100, function = "linear")
interp100_XZ = Rbf(x_XZ_subsampled100, y_XZ_subsampled100, TSE_bar_XZ_subsampled100, function = "linear")
interp100_YZ = Rbf(x_YZ_subsampled100, y_YZ_subsampled100, TSE_bar_YZ_subsampled100, function = "linear")

# Re-evaluate interpolant over original (initial) meshgrid
TSE_bar_interpolated100_xy = interp100_XY(X0_xy, Y0_xy) 
TSE_bar_interpolated100_xz = interp100_XZ(X0_xz, Z0_xz) 
TSE_bar_interpolated100_yz = interp100_YZ(Y0_yz, Z0_yz)

In [ ]:
# define minimum and maximum values for colorbar
min_value = min(np.min(TSE_bar_yz),min(np.min(TSE_bar_xy), np.min(TSE_bar_xz)))
max_value = max(np.max(TSE_bar_yz),max(np.max(TSE_bar_xy), np.max(TSE_bar_xz)))

# create figure
fig = go.Figure(data=[go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy, surfacecolor = TSE_bar_interpolated100_xy, showscale = True, colorscale='rainbow', cmin=min_value, cmax=max_value)])
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz, z = Z0_xz, surfacecolor = TSE_bar_interpolated100_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_interpolated100_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy-2*pi, surfacecolor = TSE_bar_interpolated100_xy, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz+2*pi, z = Z0_xz, surfacecolor = TSE_bar_interpolated100_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz+2*pi, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_interpolated100_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))

# camera
camera = dict(eye=dict(x=-1.45,y=-1.45,z=1.7))
fig.update_layout(scene_camera=camera)

fig.show()

In [ ]:
# 0.1% subsampling
rdn1000 = random.sample(range(0, Nx*Ny), int(Nx*Ny/1000))

# Subsample initial grid and TSE_bar field (=subsample trajectories) on XY-plane
x_XY_subsampled1000 = X0_xy.ravel()[rdn1000] # array (Nx*Ny/10, )
y_XY_subsampled1000 = Y0_xy.ravel()[rdn1000] # array (Nx*Ny/10, )
TSE_bar_XY_subsampled1000 = TSE_bar_xy.ravel()[rdn1000] # array (Nx*Ny/10, )

# Subsample initial grid and TSE_bar field (=subsample trajectories) on XZ-plane
x_XZ_subsampled1000 = X0_xz.ravel()[rdn1000] # array (Nx*Ny/10, )
y_XZ_subsampled1000 = Z0_xz.ravel()[rdn1000] # array (Nx*Ny/10, )
TSE_bar_XZ_subsampled1000 = TSE_bar_xz.ravel()[rdn1000] # array (Nx*Ny/10, )

# Subsample initial grid and TSE_bar field (=subsample trajectories) on YZ-plane
x_YZ_subsampled1000 = Y0_yz.ravel()[rdn1000] # array (Nx*Ny/10, )
y_YZ_subsampled1000 = Z0_yz.ravel()[rdn1000] # array (Nx*Ny/10, )
TSE_bar_YZ_subsampled1000 = TSE_bar_yz.ravel()[rdn1000] # array (Nx*Ny/10, )

# Compute Interpolant
interp1000_XY = Rbf(x_XY_subsampled1000, y_XY_subsampled1000, TSE_bar_XY_subsampled1000, function = "linear")
interp1000_XZ = Rbf(x_XZ_subsampled1000, y_XZ_subsampled1000, TSE_bar_XZ_subsampled1000, function = "linear")
interp1000_YZ = Rbf(x_YZ_subsampled1000, y_YZ_subsampled1000, TSE_bar_YZ_subsampled1000, function = "linear")

# Re-evaluate interpolant over original (initial) meshgrid
TSE_bar_interpolated1000_xy = interp1000_XY(X0_xy, Y0_xy) 
TSE_bar_interpolated1000_xz = interp1000_XZ(X0_xz, Z0_xz) 
TSE_bar_interpolated1000_yz = interp1000_YZ(Y0_yz, Z0_yz)

In [ ]:
# define minimum and maximum values for colorbar
min_value = min(np.min(TSE_bar_interpolated1000_xy),min(np.min(TSE_bar_interpolated1000_xy), np.min(TSE_bar_interpolated1000_xy)))
max_value = max(np.max(TSE_bar_interpolated1000_xy),max(np.max(TSE_bar_interpolated1000_xy), np.max(TSE_bar_interpolated1000_xy)))

# create figure
fig = go.Figure(data=[go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy, surfacecolor = TSE_bar_interpolated1000_xy, showscale = True, colorscale='rainbow', cmin=min_value, cmax=max_value)])
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz, z = Z0_xz, surfacecolor = TSE_bar_interpolated1000_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_interpolated1000_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy-2*pi, surfacecolor = TSE_bar_interpolated1000_xy, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz+2*pi, z = Z0_xz, surfacecolor = TSE_bar_interpolated1000_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz+2*pi, y = Y0_yz, z = Z0_yz, surfacecolor = TSE_bar_interpolated100_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))

# camera
camera = dict(eye=dict(x=-1.45,y=-1.45,z=1.7))
fig.update_layout(scene_camera=camera)

fig.show()